In [ ]:
%pip install ultralytics
%pip install opencv-python-headless matplotlib

print("Installation complete.")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Installation complete.


In [ ]:
import torch
from ultralytics import YOLO
import os

In [4]:
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"Success! Training on: {gpu_name}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB VRAM")
    device = 0
elif torch.backends.mps.is_available():
    print("Apple Silicon (MPS) detected!")
    device = "mps"
else:
    print("WARNING: GPU not found. You are running on CPU!")
    device = "cpu"

Success! Training on: NVIDIA GeForce RTX 4090
Memory: 25.25 GB VRAM


In [ ]:
dataset_yaml_path = "dataset/data.yaml"

In [ ]:
yaml_content = """
train: images/train
val: images/val
test: images/test
names:
  0: stenosis
"""
if os.path.exists("dataset"):
    with open(dataset_yaml_path, "w") as f:
        f.write(yaml_content.strip())
    print(f"Fixed data.yaml! YOLO will now use relative paths safely.")
else:
    print("WARNING: 'dataset' folder not found! Please make sure you unpacked the dataset in the same directory as this notebook.")

In [ ]:
model = YOLO("yolo26m.pt")
print("\nStarting training process...")
results = model.train(
    data="dataset/data.yaml",
    epochs=300,         
    imgsz=512,
    
    # --- SPEED OPTIMIZATIONS ---
    batch=64,
    cache=True,        
    workers=8,
    amp=True,
    patience=50,
    optimizer="AdamW",
    lr0=0.001,
    lrf=0.01,
    hsv_h=0.015,
    hsv_s=0.4,
    hsv_v=0.3,
    degrees=15,
    translate=0.1,
    scale=0.5,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    device=device,          
    project="runs/stenosis",
    plots=True,
    name="yolo26m"
)
print("Training finished!")

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu130 CUDA:0 (NVIDIA GeForce RTX 4090, 24081MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=248, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.3, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=runs/detect/runs/stenosis/yolo26m_v22/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=yolo26m_v3_512, nbs=64, nms=False, opset=None, optimize=False, optimize

In [ ]:
best_model_path = "runs/stenosis/yolo26m/weights/best.pt"

if os.path.exists(best_model_path):
    eval_model = YOLO(best_model_path)
    
    print("\nRunning Evaluation on Validation Set...")
    metrics = eval_model.val(data=dataset_yaml_path, split='val', plots=True) 
    
    # Calculate F1 Score safely
    p, r = metrics.box.mp, metrics.box.mr
    f1 = 2 * (p * r) / (p + r) if (p + r) > 0 else 0
    
    print("\n--- FINAL METRICS ---")
    print(f"mAP@50:    {metrics.box.map50:.2%}")
    print(f"mAP@50-95: {metrics.box.map:.2%}")
    print(f"Precision: {p:.2%}")
    print(f"Recall:    {r:.2%}")
    print(f"F1 Score:  {f1:.2%}")
else:
    print(f"WARNING: {best_model_path} not found. Evaluation skipped.")

In [ ]:
# --- Model Export ---
if os.path.exists(best_model_path):
    print("\nExporting model to ONNX format...")
    export_path = eval_model.export(format="onnx")
    print(f"Model successfully exported to: {export_path}")